In [1]:
import karel_env.dataset_karel as dataset

In [2]:
dataset_train, _, _ = dataset.create_default_splits('datasets/karel_dataset/', num_k=10)
data_id = dataset_train.ids
obs = dataset_train.get_data(data_id[0])

/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:168: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num_train = int(f['data_info']['num_train'].value)
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:169: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num_test = int(f['data_info']['num_test'].value)
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:170: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num_val = int(f['data_info']['num_val'].value)
[2021-12-26 17:51:44,345] Reading datasets/karel_dataset/data.hdf5 ...
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:29: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.dsl_type = self.data['data_info']['dsl_type'].value
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:30: H5p

In [ ]:
obs[2][:, :, :,:,:].shape

In [ ]:
obs[1]

In [ ]:
from karel_env import karel_util

In [ ]:
from PIL import Image
karel_util.state2symbol(dataset_train.get_data(data_id[1000])[3][4, 8, :,:,:])

In [ ]:
karel_util.state2image(dataset_train.get_data(data_id[1000])[3][3, 5, :,:,:]).shape

In [7]:
import numpy as np
pad_per_data = dataset_train.get_data(data_id[1000])[11]
def convert_to_dec(row):
    if 2 in row:
        return 32
    elif 3 in row:
        return 33
    else:
        return int(''.join(row.astype(int).astype(str)), 2)

np.apply_along_axis(convert_to_dec, 2, pad_per_data)

array([[29, 29, 29, 30, 30, 29, 30, 25, 26, 13, 14,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [29, 29, 29, 30, 30, 30, 30, 29, 30, 13, 14,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [29, 29, 29, 30, 30, 29, 30,  9, 10,  0,  0,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [21, 21, 29, 30, 30, 29, 30, 29, 30, 29, 30, 21, 22, 13, 14, 32,
        33, 33, 33, 33],
       [29, 29, 29, 30, 30, 29, 30, 13, 14,  0,  0,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [29, 29, 29, 30, 30, 29, 30, 13, 14,  0,  0,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [29, 29, 29, 30, 30, 29, 30, 29, 30, 13, 14,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [30, 30, 30, 30, 30, 29, 30, 13, 14,  0,  0,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [21, 21, 29, 30, 30, 30, 30, 29, 30, 13, 14,  0,  0,  0,  0, 32,
        33, 33, 33, 33],
       [25, 21, 29, 30, 30, 25, 26, 29, 30, 13, 14,  0,  0,  0,  0, 32,
        33, 33, 33, 33]])

In [ ]:
import matplotlib.pyplot as plt
for i in range(20):
    plt.imshow(karel_util.state2image(dataset_train.get_data(data_id[1000])[3][2, i, :,:,:]))
    plt.show()

In [1]:
import math
import numpy as np
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from torchvision import transforms
from torchmetrics import Accuracy
from karel_env.dsl import get_KarelDSL

In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [3]:
import karel_env.dataset_karel as dataset
from torch.utils.data import Dataset

class KarelVideoDataset(Dataset):
    def __init__(self, dataset_path='datasets/karel_dataset/', num_k=10, 
                 train=False, test=False, val=False):
        check_arr = [train, test, val]
        if check_arr.count(True) == 0:
            raise RuntimeError('No Dataset type specified')
        
        if check_arr.count(True) > 1:
            raise RuntimeError('Multiple Dataset types specified')
        
        dataset_train, dataset_test, dataset_val \
            = dataset.create_default_splits(dataset_path, num_k=num_k)
        self.train = train
        self.test = test
        self.val = val
        
        if self.train:
            self.data_ids = dataset_train.ids#[:128]
            self.dataset = dataset_train
        elif self.test:
            self.data_ids = dataset_test.ids#[:64]
            self.dataset = dataset_test
        else:
            self.data_ids = dataset_val.ids
            self.dataset = dataset_val

    def __len__(self):
        return len(self.data_ids)
    
    def __getitem__(self, idx):
        data_id = self.data_ids[idx]
        data = self.dataset.get_data(data_id)
        observation = data[2]
        padded_program_tokens = data[1]
        observation = torch.as_tensor(observation, dtype=torch.float32)
        demo_length = torch.as_tensor(data[9], dtype=torch.float32)
        padded_program_tokens = torch.as_tensor(padded_program_tokens, dtype=torch.long)
        padded_action_history_tokens = torch.as_tensor(data[5], dtype=torch.long)
        pad_per_data = torch.as_tensor(data[11], dtype=torch.long)
        
        
        
        demo_length = torch.as_tensor(data[9], dtype=torch.long)
        """ 
        get_data(id) returns:
            0: program, 
            1: padded_program_tokens, 
            2: demo[:self.num_k], 
            3: test_demo, 
            4: action_history[:self.num_k], 
            5: padded_action_history_tokens[:self.num_k], 
            6: test_action_history, 
            7: padded_test_action_history_tokens, 
            8: program_length
            9: demo_length[:self.num_k]
            10: test_demo_length, 
            11: pad_per_data[:self.num_k], 
            12: pad_test_per_data
            13: program tokens
        """
        return observation, padded_program_tokens, padded_action_history_tokens, pad_per_data, demo_length

In [4]:
class KarelVideoDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size

    # OPTIONAL, called for every GPU/machine (assigning state is OK)
    def setup(self, stage = None):
        # transforms
        # split dataset
        if stage in (None, "fit"):
            # in the paper they use the test for training and not the val
            # acutally the terms are somehow mixed 
            self.karel_train = KarelVideoDataset(train=True)
            self.karel_val =  KarelVideoDataset(test=True)
        if stage == "test":
            self.karel_test = KarelVideoDataset(val=True)
        if stage == "predict":
            self.karel_predict = KarelVideoDataset(val=True)

    # return the dataloader for each split
    def train_dataloader(self):
        return DataLoader(self.karel_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.karel_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.karel_test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.karel_predict, batch_size=self.batch_size)

In [5]:
PAD_IDX = 50
PAD_IDX_ACTION = 6
PAD_IDX_PERCEPTION = 33
class ProgramSynthesisTransformer(pl.LightningModule):
    def __init__(self, depth=16, w=8, h=8, k=10, max_demo_len=20, max_program_len=43):
        super().__init__()
        self.dataset_type = 'karel'
        self.dim_model = 256
        self.depth = depth
        self.k = k
        self.w = w
        self.h = h
        self.max_demo_len = max_demo_len
        self.max_program_len = max_program_len
        
        # Metrics
        self.train_program_acc = Accuracy()
        self.val_program_acc = Accuracy()
              
        """
        The CNN for encoding of the single frames of the video. Adapted from the Original Tensorflow implementation
        of the paper: Neural Program Synthesis from diverse Demonstration Videos.
        """
        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(depth, 16, kernel_size=3, stride=2, padding=5),
            nn.LeakyReLU(0.2),
            #nn.BatchNorm2d(16),
            nn.GroupNorm(1, 16),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=5),
            nn.LeakyReLU(0.2),
            #nn.BatchNorm2d(32),
            nn.GroupNorm(1, 32),
            nn.Conv2d(32, 48, kernel_size=3, stride=2, padding=5),
            nn.LeakyReLU(0.2),
            #nn.BatchNorm2d(48),
            nn.GroupNorm(1, 48),
            nn.Flatten(),
            nn.Linear(48*self.h*self.w, self.dim_model)
        )
        self.vocab_size = 50 + 1
        self.action_n = 5 + 1 + 1 # for pad token and EOS
        self.perception_n = 5
        self.perception_out_n = 34 # 0, 1 plus pad token and EOS
        self.program_embedding = nn.Embedding(self.vocab_size, self.dim_model)
        self.action_embedding = nn.Embedding(self.action_n, self.dim_model)
        self.perception_embedding = nn.Embedding(self.perception_out_n, self.dim_model)
        
        """
        Transformer Encoder for the multiple demonstrations received from the CNNs with learned positional embeddings 
        similar to the Vision Transformer.
        """
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.dim_model, nhead=8, 
                                                   batch_first=True, dropout=0.3, activation='gelu',
                                                   dim_feedforward=256)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.transformer_encoder_out = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.dim_model*self.max_demo_len, self.dim_model),
        )
        
        # position embedding is learned, similar to ViT
        self.pos_embedding = nn.Parameter(torch.randn(1, self.max_demo_len + 1, self.dim_model))
        
        """
        Program Transformer with an Encoder-Decoder Architecture for creating the program from the output of 
        the transformer_encoder. Modelling the problem in an autoregressive way, predicting each token
        after each other.
        """
        self.transformer = nn.Transformer(nhead=8, num_encoder_layers=2,
                                          num_decoder_layers=2, batch_first=True,
                                          d_model=self.dim_model, dropout=0.3,
                                          activation="gelu", dim_feedforward=256)
        self.out = nn.Linear(self.dim_model, self.vocab_size)
        
        # pos encoder for the program tokens, similar to original paper as we have tokens
        self.positional_encoder = PositionalEncoding(
            dim_model=self.dim_model, dropout_p=0.2, max_len=5000
        )
         
        """
        Action Transformer
        """
        self.action_transformer = nn.Transformer(nhead=4, num_encoder_layers=2,
                                          num_decoder_layers=2, batch_first=True,
                                          d_model=self.dim_model, dropout=0.3,
                                          activation="gelu", dim_feedforward=128)
        self.action_out = nn.Linear(self.dim_model, self.action_n)
        
        """
        Perception Transformer
        """
        self.perception_transformer = nn.Transformer(nhead=4, num_encoder_layers=2,
                                          num_decoder_layers=2, batch_first=True,
                                          d_model=self.dim_model, dropout=0.3,
                                          activation="gelu", dim_feedforward=128)
        self.perception_out = nn.Linear(self.dim_model, self.perception_out_n)
        
        """
        Use Xavier Initialization for the weights.
        """
        for p in self.transformer.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
        for p in self.action_transformer.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
        for p in self.perception_transformer.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                        
        for p in self.transformer_encoder.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    # from: https://towardsdatascience.com/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    def get_tgt_mask(self, tgt, pad_idx=None) -> torch.tensor:
        size = tgt.size(1)
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        if pad_idx == None:
            pad_idx = PAD_IDX
        tgt_padding_mask = (tgt == pad_idx)
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask.to(self.device), tgt_padding_mask
    
    def get_src_padding_mask(self, demo_length):
        mask = torch.ones((demo_length.size(0), demo_length.size(1), self.max_demo_len))
        
        for i in range(demo_length.size(0)):
            for j in range(demo_length.size(1)):
                mask[i, j, :demo_length[i][j]] = 0
        mask = mask.bool()
        return mask.to(self.device)
        
    
    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]
        return (matrix == pad_token)

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        embedding = self.encoder(x)
        return embedding
    
    def stack_demos(self, x, batch_size, demo_length, src_pad_mask):
        # (32, 10, 20, 8, 8, 16)
        # (batch_size, demos, demo_len, w, h, depth)
        demo_tensors = None
        demo_list = []
        for i in range(self.k):
            s_h = x[:, i, :, :, :, :]
            s_h = s_h.reshape([-1, self.h, self.w, self.depth]).permute(0, 3, 1, 2)
            s_h = self.cnn_encoder(s_h)
            s_h = s_h.reshape(batch_size, self.max_demo_len, -1)
            s_h += self.pos_embedding[:, :self.max_demo_len]
            # we add it for the perception and action transformer
            demo_list.append(s_h)
            s_t = self.transformer_encoder(s_h, src_key_padding_mask=src_pad_mask[:, i, :])
            s_t = self.transformer_encoder_out(s_t)
            if demo_tensors is None:
                demo_tensors = s_t.unsqueeze(0)
            else:
                demo_tensors = torch.cat([demo_tensors, s_t.unsqueeze(0)])
        
        demo_tensors = demo_tensors.permute(1, 0, 2)
        return demo_tensors, torch.stack(demo_list).to(self.device)
    
    def run_program_transformer(self, y, demo_tensors):
        emb_program = self.program_embedding(y)
        emb_program = self.positional_encoder(emb_program)
        tgt_mask, pad_mask = self.get_tgt_mask(y)
        output = self.transformer(demo_tensors, emb_program, tgt_mask=tgt_mask, tgt_key_padding_mask=pad_mask)
        output = self.out(output)
        output = F.softmax(output, dim=-1)
        output = output.permute(0, 2, 1)
        return output
    
    def run_action_transformer(self, actions, demo_tensors, demo_length, src_pad_mask):
        outputs = []
        loss = 0.0
        for i in range(self.k):
            y = actions[:, i, :]
            demo = demo_tensors[i]
            emb_actions = self.action_embedding(y)
            emb_actions = self.positional_encoder(emb_actions)
            tgt_mask, pad_mask = self.get_tgt_mask(y, pad_idx=PAD_IDX_ACTION)
            output = self.action_transformer(demo, emb_actions, src_key_padding_mask=src_pad_mask[:, i, :], tgt_mask=tgt_mask, tgt_key_padding_mask=pad_mask)
            output = self.action_out(output)
            output = F.softmax(output, dim=-1)
            output = output.permute(0, 2, 1)
            loss += F.cross_entropy(output, y, ignore_index=PAD_IDX_ACTION)
            outputs.append(output)
        return outputs, loss / self.k 
    
            
    def run_perception_transformer(self, perceptions, demo_tensors, src_pad_mask):
        outputs = []
        loss = 0.0
        for i in range(self.k):
            seq = []
            y = perceptions[:, i, :]
            demo = demo_tensors[i]
            emb_perceptions = self.perception_embedding(y)
            emb_perceptions = self.positional_encoder(emb_perceptions)
            tgt_mask, pad_mask = self.get_tgt_mask(y, pad_idx=PAD_IDX_PERCEPTION)
            output = self.perception_transformer(demo, emb_perceptions, src_key_padding_mask=src_pad_mask[:, i, :], tgt_mask=tgt_mask, tgt_key_padding_mask=pad_mask)
            output = self.perception_out(output)
            output = F.softmax(output, dim=-1)
            output = output.permute(0, 2, 1)
            loss += F.cross_entropy(output, y, ignore_index=PAD_IDX_PERCEPTION)
            outputs.append(output)
        return outputs, loss / self.k
    
    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y, actions_inp, perceptions_inp, demo_length = batch
        batch_size = len(batch[0])
        src_pad_mask = self.get_src_padding_mask(demo_length)
        demo_summary_tensor, demo_list = self.stack_demos(x, batch_size, demo_length, src_pad_mask)
        #output = self.run_program_transformer(y, demo_summary_tensor)
        #actions, action_loss = self.run_action_transformer(actions_inp, demo_list, demo_length, src_pad_mask)
        perceptions, perception_loss = self.run_perception_transformer(perceptions_inp, demo_list, src_pad_mask)
        
        # Logging to TensorBoard by default
        #program_loss = F.cross_entropy(output, y, ignore_index=PAD_IDX)
        #self.log("train_program_loss", program_loss)
        #self.train_program_acc(output, y)
        #self.log('train_program_acc', self.train_program_acc, on_step=True, on_epoch=False)
        
        #self.log("train_action_loss", action_loss)
        #self.log("train_perception_loss", perception_loss)
        #return program_loss + action_loss + perception_loss
        return perception_loss
    
    def validation_step(self, batch, batch_idx):
        x, y, actions_inp, perceptions_inp, demo_length = batch
        #print('actions', actions_inp[1])
        #print('perceptions', perceptions_inp[0])
        batch_size = len(batch[0])
        src_pad_mask = self.get_src_padding_mask(demo_length)
        demo_summary_tensor, demo_list = self.stack_demos(x, batch_size, demo_length, src_pad_mask)
        #output = self.run_program_transformer(y, demo_summary_tensor)
        #actions, action_loss = self.run_action_transformer(actions_inp, demo_list, demo_length, src_pad_mask)
        perceptions, perception_loss = self.run_perception_transformer(perceptions_inp, demo_list, src_pad_mask)
        
        if batch_idx == 0:
            print()
            #print('  GT:', y[0])
            #print('PRED:', torch.argmax(output[0], dim=1))
            print('  GT:', perceptions_inp[0][0])
            print('PRED:', torch.argmax(perceptions[0], dim=1)[0])
            
        # Logging to TensorBoard by default
        #program_loss = F.cross_entropy(output, y, ignore_index=PAD_IDX)
        #self.log("val_program_loss", program_loss)
        #self.val_program_acc(output, y)
        #self.log('val_program_acc', self.val_program_acc, on_step=True, on_epoch=False)
        
        #self.log("val_action_loss", action_loss)
        #self.log("val_perception_loss", perception_loss)
        
    def test_step(self, batch, batch_idx):
        x, y, program_len = batch
        batch_size = len(batch[0])
        demo_tensors = self.stack_demos(x, batch_size)
        output = self.run_program_transformer(y, demo_tensors)
        
        if batch_idx == 0:
            for i in range(batch_size):
                print('GT:', y[i].shape)
                print('PRED:', torch.argmax(output[i], dim=1).shape)
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        #optimizer = torch.optim.Adam(self.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
        return optimizer

In [ ]:
# init model
model = ProgramSynthesisTransformer()
train_loader = KarelVideoDataModule(batch_size=64)
# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(gpus=1, gradient_clip_val=0.5, stochastic_weight_avg=True)
trainer.fit(model, train_loader)

/home/eberhardinger/anaconda3/envs/python3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:58: LightningDeprecationWarning: Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:169: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num_train = int(f['data_info']['num_train'].value)
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:170: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  num_test = int(f['data_info']['num_test'].value)
/home/eberhardinger/workspaces/demo2

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/eberhardinger/anaconda3/envs/python3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:47: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  program_tokens = self.data[id]['program'].value
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:53: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  demo_data = self.data[id]['s_h'].value
/home/eberhardinger/workspaces/demo2program/karel_env/dataset_karel.py:54: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  test_demo_data = self.data[id]


  GT: tensor([29, 29, 29, 29, 29,  9,  5,  6,  6,  6,  6, 31, 31, 31, 31, 31, 31, 31,
        31, 31], device='cuda:0')
PRED: tensor([18, 18, 18, 18, 18, 24, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20], device='cuda:0')
                                                                      

/home/eberhardinger/anaconda3/envs/python3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:116: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  83%|████████▎ | 391/470 [05:43<01:09,  1.14it/s, loss=2.96, v_num=0]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/79 [00:00<?, ?it/s]
  GT: tensor([29, 29, 29, 29, 29,  9,  5,  6,  6,  6,  6, 31, 31, 31, 31, 31, 31, 31,
        31, 31], device='cuda:0')
PRED: tensor([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 31, 31, 31, 31, 31, 31, 31,
        31, 31], device='cuda:0')

Epoch 1:  70%|███████   | 329/470 [04:45<02:02,  1.15it/s, loss=2.72, v_num=0]

In [ ]:
trainer = pl.Trainer(gpus=1, resume_from_checkpoint='lightning_logs/version_5/checkpoints/epoch=5-step=2345.ckpt')
trainer.test(model=model, datamodule=train_loader)

In [ ]:
from karel_env.dsl import get_KarelDSL
vocab = get_KarelDSL(dsl_type=dataset_train.dsl_type.decode(), seed=123)
vocab.__dict__